# <center> Mlflow Walkthrough

In [1]:
import warnings

import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.linear_model import ElasticNet

import mlflow
import mlflow.sklearn

### The data

In [2]:
data_path = "wine-quality.csv"
data = pd.read_csv(data_path)

data.sample(10)

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
2634,8.7,0.31,0.73,14.35,0.044,27.0,191.0,1.00013,2.96,0.88,8.7,5
4856,7.1,0.23,0.39,13.70,0.058,26.0,172.0,0.99755,2.90,0.46,9.0,6
3622,7.2,0.24,0.30,1.20,0.037,11.0,95.0,0.98914,2.96,0.36,12.5,6
146,5.6,0.16,0.27,1.40,0.044,53.0,168.0,0.99180,3.28,0.37,10.1,6
939,7.7,0.24,0.29,15.30,0.044,39.0,194.0,0.99820,3.06,0.47,9.6,7
3397,6.3,0.19,0.29,5.50,0.042,44.0,189.0,0.99304,3.19,0.47,10.3,6
2232,8.4,0.20,0.31,2.80,0.054,16.0,89.0,0.99416,2.96,0.45,9.5,6
762,6.8,0.24,0.49,19.30,0.057,55.0,247.0,1.00055,3.00,0.56,8.7,5
1733,8.1,0.30,0.31,1.10,0.041,49.0,123.0,0.99140,2.99,0.45,11.1,6
6,6.2,0.32,0.16,7.00,0.045,30.0,136.0,0.99490,3.18,0.47,9.6,6


## 1. Tracking experiments

In [3]:
remote_server_uri = "http://127.0.0.1:8000"
mlflow.set_tracking_uri(remote_server_uri)

In [4]:
mlflow.tracking.get_tracking_uri()

'http://127.0.0.1:8000'

In [5]:
exp_name = "ElasticNet_wine"
mlflow.set_experiment(exp_name)

2024/07/24 16:47:18 INFO mlflow.tracking.fluent: Experiment with name 'ElasticNet_wine' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/552358686902198705', creation_time=1721819838645, experiment_id='552358686902198705', last_update_time=1721819838645, lifecycle_stage='active', name='ElasticNet_wine', tags={}>

In [6]:
def eval_metrics(actual, pred):
    # compute relevant metrics
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2


def load_data(data_path):
    data = pd.read_csv(data_path)

    # Split the data into training and test sets. (0.75, 0.25) split.
    train, test = train_test_split(data)

    # The predicted column is "quality" which is a scalar from [3, 9]
    train_x = train.drop(["quality"], axis=1)
    test_x = test.drop(["quality"], axis=1)
    train_y = train[["quality"]]
    test_y = test[["quality"]]
    return train_x, train_y, test_x, test_y


def train(alpha=0.5, l1_ratio=0.5):
    # train a model with given parameters
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Read the wine-quality csv file (make sure you're running this from the root of MLflow!)
    data_path = "wine-quality.csv"
    train_x, train_y, test_x, test_y = load_data(data_path)

    # Useful for multiple runs (only doing one run in this sample notebook)
    with mlflow.start_run():
        # Execute ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param(key="alpha", value=alpha)
        mlflow.log_param(key="l1_ratio", value=l1_ratio)
        mlflow.log_metric(key="rmse", value=rmse)
        mlflow.log_metrics({"mae": mae, "r2": r2})
        mlflow.log_artifact(data_path)
        print("Save to: {}".format(mlflow.get_artifact_uri()))

        mlflow.sklearn.log_model(lr, "model")

In [7]:
train(0.5, 0.5)

Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160691
  R2: 0.12678721972772689
Save to: mlflow-artifacts:/552358686902198705/bdb355f1a2f14af0bc14c6325b519ece/artifacts


In [8]:
train(0.2, 0.2)

Elasticnet model (alpha=0.200000, l1_ratio=0.200000):
  RMSE: 0.7859129997062342
  MAE: 0.6155290394093895
  R2: 0.2022463182289208
Save to: mlflow-artifacts:/552358686902198705/5a23660ff2604470b1f7d33043a91bef/artifacts


In [9]:
train(0.1, 0.1)

Elasticnet model (alpha=0.100000, l1_ratio=0.100000):
  RMSE: 0.7792546522251949
  MAE: 0.6112547988118587
  R2: 0.2157063843066196
Save to: mlflow-artifacts:/552358686902198705/aa1790e0bd724eea9119c11b607c7f07/artifacts


In [10]:
train(0.2, 0.3)

Elasticnet model (alpha=0.200000, l1_ratio=0.300000):
  RMSE: 0.7878483325956936
  MAE: 0.6165599984451102
  R2: 0.1983124988099163
Save to: mlflow-artifacts:/552358686902198705/ce354ac7af2543de864699593cee865b/artifacts


In [11]:
train(0.3, 0.4)

Elasticnet model (alpha=0.300000, l1_ratio=0.400000):
  RMSE: 0.7982627137388201
  MAE: 0.6208751951322825
  R2: 0.17697777967442674
Save to: mlflow-artifacts:/552358686902198705/98f2007d200449a6b8ebeec53097af1d/artifacts
